In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Updated to match my CRUD Python module file name and class name
from project_1 import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Added necessary variables and values to connect to database
username = "aacuser"
password = "BlueGold0120"
host = 'nv-desktop-services.apporto.com'
port = 31860
db = 'AAC'
collection = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, db, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Added Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    # Added unique identifier to H1 tag
    html.Center(html.B(html.H1('CS-340 Dashboard - Taylor Colton'))),
    # Added HTML Img tag for Grazioso Salvare logo
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                    'display': 'block',
                    'margin-left': 'auto',
                    'margin-right': 'auto',
                    'width': '200px',
                    'height': '200px'
            }),
    html.Hr(),
    # Drop own menu for interactive filtering
    html.Div(dcc.Dropdown([
                            'Water Rescue',
                            'Mountain or Wilderness Rescue',
                            'Disaster or Individual Tracking',
                            'Reset'],
                            id='filter-type'),
    ),
    # User-friendly interactive data table
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable="single",  # Allow single row selection
                         selected_rows=[0],
                         selected_columns=[],
                         style_data_conditional=[],
                         style_table={'overflowX': 'auto'},
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id','data'),
              Output('datatable-id', 'columns'),
              Output('datatable-id', 'selected_rows'),
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):
    """Filters interactive data table with MongoDB queries"""
    if filter_type == "Water Rescue":
        df = pd.DataFrame.from_records(db.read({
            "breed": {"$in": [
                "Labrador Retriever Mix", 
                "Chesapeake Bay Retriever", 
                "Newfoundland"
            ]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        })) 
    elif filter_type == "Mountain or Wilderness Rescue":
        df = pd.DataFrame.from_records(db.read({
            "breed": {"$in": [
                "German Shepherd", 
                "Alaskan Malamute", 
                "Old English Sheepdog", 
                "Siberian Husky", 
                "Rottweiler"
            ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        })) 
    elif filter_type == "Disaster or Individual Tracking":
        df = pd.DataFrame.from_records(db.read({
            "breed": {"$in": [
                "Doberman Pinscher", 
                "German Shepherd", 
                "Golden Retriever",
                "Bloodhound", 
                "Rottweiler"
            ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        })) 
    else:
        df = pd.DataFrame.from_records(db.read({}))
    
    df.drop(columns=['_id'],inplace=True)
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data,columns,[0])

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    """Updates pie chart with filtered and selected data"""
    if viewData is None:
        return html.Div("No data available to display.")
    
    # Convert viewData to DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Count occurrences of each breed
    breed_counts = dff['breed'].value_counts(normalize=True) * 100  # Convert to percentages

    # Separate breeds based on percentage
    main_breeds = breed_counts[breed_counts >= 1]  # Breeds >= 1%
    other_breeds = breed_counts[breed_counts < 1]   # Breeds < 1%

    # Sum up the "Other" breeds
    other_count = other_breeds.sum()

    # Create a new DataFrame for the pie chart
    chart_data = pd.concat([main_breeds, pd.Series({'Other': other_count})])

    return [
        dcc.Graph(            
            figure = px.pie(chart_data, names=chart_data.index, values=chart_data.values, title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    """Updates map based on data in selected row"""
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection,
    # the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Selected animal location:
    selected_row = viewData[row]
    latitude = selected_row.get("location_lat")
    longitude = selected_row.get("location_long")
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[latitude,longitude], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:20318/
